In [1]:
import torch
import numpy as np

from utils import Dataset, setup_seed, get_edge_index
from models import NeuMF, ConvNCF, BPR, LightGCN
from train import train

D:\Program Files (x86)\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
D:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
dataset_path = 'data/lastfm'

# Load the dataset
dataset = Dataset(dataset_path)
train_mat, test_ratings, test_negatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
print('Dataset: #user=%d, #item=%d, #train_pairs=%d, #test_pairs=%d' 
      % (dataset.num_users, dataset.num_items, train_mat.nnz, len(test_ratings)))

mat = train_mat.todense()
mat = 1 * (mat > 0)
if type(mat) is np.matrix: mat = np.asarray(mat)

Dataset: #user=518, #item=3488, #train_pairs=45654, #test_pairs=518


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

users_num, items_num = dataset.num_users, dataset.num_items

# BPR

In [4]:
lr = 0.001
weight_decay = 1e-6
embedding_size = 64
epochs = 100
batch_size = 1024
mode = 'hr'
topK = 10
Loss = 'BPR'

hr_list, ndcg_list = [], []
for seed in range(4):
    setup_seed(seed)
    model = BPR(users_num, items_num, embedding_size).to(device)
    hr, ndcg = train(model=model, 
                     mat=mat, 
                     test_ratings=test_ratings, 
                     test_negatives=test_negatives, 
                     lr=lr, 
                     weight_decay=weight_decay,
                     batch_size=batch_size,
                     topK=topK, 
                     epochs=epochs, 
                     mode=mode, 
                     device=device,
                     Loss=Loss)

    hr_list.append(hr), ndcg_list .append(ndcg)
    print("End. Best HR = %.4f, NDCG = %.4f. " %(hr, ndcg))
    print('-' * 100)
print('HR: %.4f' %np.mean(hr_list), 'NDCG: %.4f' %np.mean(ndcg_list))
print('=' * 100)

Init: HR = 0.1120, NDCG = 0.0473
epoch=0, loss=414.3769, HR=0.2201, NDCG=0.1089
epoch=1, loss=411.3254, HR=0.6178, NDCG=0.4144
epoch=2, loss=395.5713, HR=0.6409, NDCG=0.4499
epoch=3, loss=360.0828, HR=0.6390, NDCG=0.4578
epoch=4, loss=312.9675, HR=0.6448, NDCG=0.4625
epoch=5, loss=277.1581, HR=0.6525, NDCG=0.4732
epoch=6, loss=255.6710, HR=0.6737, NDCG=0.4800
epoch=7, loss=248.1858, HR=0.6853, NDCG=0.4916
epoch=8, loss=213.7754, HR=0.7046, NDCG=0.5026
epoch=9, loss=194.1738, HR=0.7066, NDCG=0.5056
epoch=10, loss=197.9709, HR=0.7201, NDCG=0.5153
epoch=11, loss=182.1439, HR=0.7278, NDCG=0.5202
epoch=12, loss=167.3978, HR=0.7239, NDCG=0.5258
epoch=13, loss=176.4538, HR=0.7317, NDCG=0.5304
epoch=14, loss=170.3770, HR=0.7297, NDCG=0.5303
epoch=15, loss=146.9813, HR=0.7375, NDCG=0.5376
epoch=16, loss=132.7195, HR=0.7355, NDCG=0.5398
epoch=17, loss=130.7808, HR=0.7394, NDCG=0.5476
epoch=18, loss=130.3804, HR=0.7413, NDCG=0.5519
epoch=19, loss=138.7411, HR=0.7490, NDCG=0.5550
epoch=20, loss=11

### NeuMF

In [6]:
lr = 0.01
weight_decay = 1e-6
embedding_size = 16
num_layers = 1
epochs = 32
batch_size = 1024
mode = 'hr'
topK = 10
Loss = 'BCE'

hr_list, ndcg_list = [], []
for seed in range(4):
    setup_seed(seed)
    model = NeuMF(users_num, items_num, embedding_size, num_layers)
    model = model.to(device)
    hr, ndcg = train(model=model, 
                     mat=mat, 
                     test_ratings=test_ratings, 
                     test_negatives=test_negatives, 
                     lr=lr, 
                     weight_decay=weight_decay,
                     batch_size=batch_size,
                     topK=topK, 
                     epochs=epochs, 
                     mode=mode, 
                     device=device,
                     Loss=Loss)
    hr_list.append(hr), ndcg_list .append(ndcg)
    print("End. Best HR = %.4f, NDCG = %.4f. " %(hr, ndcg))
    print('-' * 100)
print('HR: %.4f' %np.mean(hr_list), 'NDCG: %.4f' %np.mean(ndcg_list))
print('=' * 100)

Init: HR = 0.1081, NDCG = 0.0481
epoch=0, loss=0.4091, HR=0.6274, NDCG=0.4318
epoch=1, loss=0.2853, HR=0.7104, NDCG=0.4924
epoch=2, loss=0.2892, HR=0.7355, NDCG=0.5114
epoch=3, loss=0.2747, HR=0.7432, NDCG=0.5126
epoch=4, loss=0.2416, HR=0.7317, NDCG=0.5291
epoch=5, loss=0.2273, HR=0.7471, NDCG=0.5174
epoch=6, loss=0.2479, HR=0.7664, NDCG=0.5244
epoch=7, loss=0.2367, HR=0.7452, NDCG=0.5197
epoch=8, loss=0.1843, HR=0.7432, NDCG=0.5214
End. Best HR = 0.7664, NDCG = 0.5244. 
----------------------------------------------------------------------------------------------------
Init: HR = 0.1062, NDCG = 0.0486
epoch=0, loss=0.4100, HR=0.5965, NDCG=0.4197
epoch=1, loss=0.3837, HR=0.7027, NDCG=0.5008
epoch=2, loss=0.2993, HR=0.7239, NDCG=0.5148
epoch=3, loss=0.2580, HR=0.7297, NDCG=0.5214
epoch=4, loss=0.2585, HR=0.7432, NDCG=0.5237
epoch=5, loss=0.2007, HR=0.7201, NDCG=0.5112
epoch=6, loss=0.2093, HR=0.7201, NDCG=0.5069
End. Best HR = 0.7432, NDCG = 0.5237. 
-----------------------------------

### ConvNCF

In [7]:
lr = 0.001
weight_decay = 1e-6
embedding_size = 64
epochs = 32
batch_size = 1024
mode = 'hr'
topK = 10
Loss = 'BPR'

hr_list, ndcg_list = [], []
for seed in range(4):
    setup_seed(seed)
    model = ConvNCF(users_num=users_num, items_num=items_num, embedding_size=embedding_size, device=device)
    model = model.to(device)
    hr, ndcg = train(model=model, 
                     mat=mat, 
                     test_ratings=test_ratings, 
                     test_negatives=test_negatives, 
                     lr=lr, 
                     weight_decay=weight_decay,
                     batch_size=batch_size,
                     topK=topK, 
                     epochs=epochs, 
                     mode=mode, 
                     device=device,
                     Loss=Loss)
    hr_list.append(hr), ndcg_list .append(ndcg)
    print("End. Best HR = %.4f, NDCG = %.4f. " %(hr, ndcg))
    print('-' * 100)
print('HR: %.4f' %np.mean(hr_list), 'NDCG: %.4f' %np.mean(ndcg_list))
print('=' * 100)

Init: HR = 0.1042, NDCG = 0.0519
epoch=0, loss=414.5019, HR=0.2162, NDCG=0.1208
epoch=1, loss=412.2173, HR=0.3745, NDCG=0.2250
epoch=2, loss=319.4777, HR=0.4923, NDCG=0.3130
epoch=3, loss=269.9120, HR=0.5463, NDCG=0.3520
epoch=4, loss=253.3635, HR=0.5714, NDCG=0.3684
epoch=5, loss=227.1263, HR=0.6371, NDCG=0.4270
epoch=6, loss=187.4999, HR=0.6660, NDCG=0.4569
epoch=7, loss=137.5470, HR=0.6988, NDCG=0.4781
epoch=8, loss=124.3939, HR=0.7124, NDCG=0.4884
epoch=9, loss=137.3982, HR=0.7085, NDCG=0.4971
epoch=10, loss=119.1349, HR=0.7143, NDCG=0.5045
epoch=11, loss=108.1831, HR=0.7124, NDCG=0.5057
epoch=12, loss=85.4030, HR=0.7143, NDCG=0.5126
End. Best HR = 0.7143, NDCG = 0.5045. 
----------------------------------------------------------------------------------------------------
End. Best HR = 0.7143, NDCG = 0.5045. 
----------------------------------------------------------------------------------------------------
Init: HR = 0.1081, NDCG = 0.0506
epoch=0, loss=414.4998, HR=0.3571, NDCG=0

# LightGCN

In [4]:
lr = 0.001
weight_decay = 1e-8
embedding_size = 128
epochs = 100
batch_size = 1024
mode = 'hr'
topK = 10
Loss = 'BPR'

edge_index = get_edge_index(mat)

hr_list, ndcg_list = [], []
for seed in range(4):
    setup_seed(seed)
    model = LightGCN(users_num=users_num, items_num=items_num, embedding_size=embedding_size, edge_index=edge_index).to(device)
    hr, ndcg = train(model=model, 
                     mat=mat, 
                     test_ratings=test_ratings, 
                     test_negatives=test_negatives, 
                     lr=lr, 
                     weight_decay=weight_decay,
                     batch_size=batch_size,
                     topK=topK, 
                     epochs=epochs, 
                     mode=mode, 
                     device=device,
                     Loss=Loss)

    hr_list.append(hr), ndcg_list .append(ndcg)
    print("End. Best HR = %.4f, NDCG = %.4f. " %(hr, ndcg))
    print('-' * 100)
print('HR: %.4f' %np.mean(hr_list), 'NDCG: %.4f' %np.mean(ndcg_list))
print('=' * 100)

Init: HR = 0.0927, NDCG = 0.0411
epoch=0, loss=182.7557, HR=0.5425, NDCG=0.3769
epoch=1, loss=119.1079, HR=0.6313, NDCG=0.4454
epoch=2, loss=105.7722, HR=0.6834, NDCG=0.4855
epoch=3, loss=102.9729, HR=0.7143, NDCG=0.5033
epoch=4, loss=85.2205, HR=0.7066, NDCG=0.5103
epoch=5, loss=65.1514, HR=0.7143, NDCG=0.5126
End. Best HR = 0.7143, NDCG = 0.5033. 
----------------------------------------------------------------------------------------------------
End. Best HR = 0.7143, NDCG = 0.5033. 
----------------------------------------------------------------------------------------------------
HR: 0.7143 NDCG: 0.5033
